## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", index_col=0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
City_ID,,,,,,,,,
0,Kropotkin,RU,45.4375,40.5756,59.14,89,97,6.55,overcast clouds
1,Bethel,US,41.3712,-73.4140,75.72,73,48,7.94,scattered clouds
2,Punta Arenas,CL,-53.1500,-70.9167,50.95,75,100,15.88,overcast clouds
3,Jamestown,US,42.0970,-79.2353,75.18,61,100,5.99,overcast clouds
4,Calabozo,VE,8.9242,-67.4293,85.82,67,99,5.35,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria ##MY SETTING SET AT 65 - 85
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   323
Country                321
Lat                    323
Lng                    323
Max Temp               323
Humidity               323
Cloudiness             323
Wind Speed             323
Current Description    323
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df= preferred_cities_df.dropna()
preferred_cities_df.count()

City                   321
Country                321
Lat                    321
Lng                    321
Max Temp               321
Humidity               321
Cloudiness             321
Wind Speed             321
Current Description    321
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
1,Bethel,US,75.72,scattered clouds,41.3712,-73.4140,
3,Jamestown,US,75.18,overcast clouds,42.0970,-79.2353,
6,Foggia,IT,68.77,broken clouds,41.4500,15.5333,
9,Albany,US,74.80,overcast clouds,42.6001,-73.9662,
14,Mahebourg,MU,70.25,scattered clouds,-20.4081,57.7000,
15,Kondoa,TZ,67.95,broken clouds,-4.9000,35.7833,
16,Rikitea,PF,73.63,clear sky,-23.1203,-134.9692,
18,Puerto Ayora,EC,73.54,broken clouds,-0.7393,-90.3518,
19,Medicine Hat,CA,67.21,clear sky,50.0501,-110.6683,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"    

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotels = requests.get(base_url, params=params).json()    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace("",np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [10]:
clean_hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
1,Bethel,US,75.72,scattered clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
3,Jamestown,US,75.18,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
6,Foggia,IT,68.77,broken clouds,41.4500,15.5333,B&B White
14,Mahebourg,MU,70.25,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
15,Kondoa,TZ,67.95,broken clouds,-4.9000,35.7833,Kinyuma Lodge
16,Rikitea,PF,73.63,clear sky,-23.1203,-134.9692,People ThankYou
18,Puerto Ayora,EC,73.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
19,Medicine Hat,CA,67.21,clear sky,50.0501,-110.6683,Sun Dek Motel
23,Faanui,PF,78.12,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))